In [6]:
import torch
from torch import nn
from torch.optim import SGD
from torch import Tensor
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [7]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [8]:
def train_loop(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss, optimizer: SGD) -> None:
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred: Tensor = model(X)
        loss: Tensor = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader: DataLoader, model: NeuralNetwork, loss_fn: nn.CrossEntropyLoss) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred: Tensor = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
loss: 2.304151  [    0/60000]
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.Size([64, 10])
torch.

[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [10]:
print(torch.__version__)

1.13.0
